**Imports and set magic**

In [6]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import os
import numpy as np
from scipy.optimize import minimize
from scipy.io import loadmat
from Likelihood import Likelihood
from functools import partial

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


**Folder path definition: Copy and insert local path of "1 Summary Statistics/Data"**

In [7]:
folder = "/Users/emmaknippel/Desktop/POLIT 10 SEM/Dynamic Programming/Replicating the Innovators Dilemma/1 Summary Statistics/Data"

**Loading the relevant data**

In [13]:
def load_csv(filename, folder):
    return pd.read_csv(os.path.join(folder, f"{filename}.csv"), header=0).values

State = load_csv("State", folder)
# Ensure that there are 4 potential entrants in each period
State[:,3] = np.full(State.shape[0], 4.0)
Adopt = load_csv("Adopt", folder)
Exit = load_csv("Exit", folder)

Pi = loadmat('CheckPi.mat')['Pi']

**Calling the Likelihood function for given dynamic parameter values**

In [15]:
T = Pi.shape[0]
beta = 0.80      # Discount factor
delta = 1.10   # Decline in SC
scaling = 10000   # Scaling factor
iterMLE = 1

# Rescale profits
Pi = Pi / scaling

# Terminal Values
V = np.zeros((T, 3, 12, 12, 15))
V[T-1,:,:,:,:] = Pi[T-1,:,:,:,:] / (1 - beta)

# Initialize Solutions
EV = np.zeros((T, 5, 12, 12, 15))
Policy = np.zeros((T-1, 5, 12, 12, 15))

Theta = [-0.1473624217307790, 1.24394591815957, 2.25381667717391]

logL = Likelihood(Theta=Theta, beta=beta, delta=delta, Pi=Pi, V=V, EV=EV, Policy=Policy, State=State, Exit=Exit, Adopt=Adopt, T=T, iterMLE=iterMLE, output_type=2)


MLE iter #   1: trying (phi, kappa_inc, kappa_ent) = (-0.14736242, 1.24394592, 2.25381668)


KeyboardInterrupt: 

**Running MLE for starting values x0 = [1, 1, 1]**

In [3]:
T = Pi.shape[0]
beta = 0.80      # Discount factor
delta = 1.10   # Decline in SC
scaling = 10000   # Scaling factor
iterMLE = 1

print(f"  -----------------------------------------")
print(f" | STRATEGIC (RATIONAL-EXPECTATIONS) MODEL |")
print(f"  -----------------------------------------")

print(f" ----- Basic parameters to be \"calibrated\" -----")
print(f"   Profit/value scaling      :     {scaling:4.0f}")
print(f"   Discount factor (beta)    :     {beta:1.4f}")
print(f"   Adoption cost drop (delta):     {delta:1.4f}")
print(" -----------------------------------------------\n")

# Rescale profits
Pi = Pi / scaling

# Terminal Values
V = np.zeros((T, 3, 12, 12, 15))
V[T-1,:,:,:,:] = Pi[T-1,:,:,:,:] / (1 - beta)

# Initialize Solutions
EV = np.zeros((T, 5, 12, 12, 15))
Policy = np.zeros((T-1, 5, 12, 12, 15))

# MLE Estimation
x0 = [1, 1, 1]
options = {'disp': True, 'maxiter': 1000, 'xatol': 1e-4, 'fatol': 1e-8}

objective_function = partial(Likelihood, beta=beta, delta=delta, Pi=Pi, V=V, EV=EV, Policy=Policy, State=State, Exit=Exit, Adopt=Adopt, T=T, iterMLE=iterMLE, output_type=1)

res = minimize(objective_function, x0, method='Nelder-Mead', options=options)
Theta = res.x

# Numerical derivatives for standard errors
perturb = 0.001
logL = Likelihood(Theta, 2)
logLderiv = np.zeros((1, 3))

for k in range(3):
    Theta_perturbed = np.copy(Theta)
    Theta_perturbed[k] += perturb
    logL_perturbed = Likelihood(Theta_perturbed, 2)
    logLderiv[0, k] = (logL_perturbed - logL) / perturb

# Derive the Value estimate
Vhat = np.linalg.inv(logLderiv.T @ logLderiv)

print(f"\n ----- Maximum Likelihood Estimation Results for (phi & kappa_inc) -----")
print(f" Coeff.: " + " ".join([f"{v:8.8f}" for v in Theta]))
print(f" S.E.  : " + " ".join([f"{v:8.8f}" for v in np.sqrt(np.diag(Vhat))]))
print(f" -----------------------------------------------------------------------\n")

  -----------------------------------------
 | STRATEGIC (RATIONAL-EXPECTATIONS) MODEL |
  -----------------------------------------
 ----- Basic parameters to be "calibrated" -----
   Profit/value scaling      :     10000
   Discount factor (beta)    :     0.8000
   Adoption cost drop (delta):     1.1000
 -----------------------------------------------


MLE iter #   1: trying (phi, kappa_inc, kappa_ent) = (1.00000000, 1.00000000, 1.00000000)


/Users/emmaknippel/Desktop/POLIT 10 SEM/Dynamic Programming/Term Paper/ReplicatingInnovatorsDilemma/3 Maximum Likelihood/fun8.py:16: RuntimeWarning: overflow encountered in exp
  stay = np.exp(phi + beta * z3)
/Users/emmaknippel/Desktop/POLIT 10 SEM/Dynamic Programming/Term Paper/ReplicatingInnovatorsDilemma/3 Maximum Likelihood/fun12.py:15: RuntimeWarning: overflow encountered in exp
  stay = np.exp(phi + beta * z3)
/Users/emmaknippel/Desktop/POLIT 10 SEM/Dynamic Programming/Term Paper/ReplicatingInnovatorsDilemma/3 Maximum Likelihood/fun6.py:19: RuntimeWarning: overflow encountered in exp
  adopt = np.exp(phi + beta * z2 - kappa_inc * (delta ** (year - 1)))
/Users/emmaknippel/Desktop/POLIT 10 SEM/Dynamic Programming/Term Paper/ReplicatingInnovatorsDilemma/3 Maximum Likelihood/fun7.py:20: RuntimeWarning: overflow encountered in exp
  adopt = np.exp(phi + beta * z2 - kappa_inc * (delta ** (year-1)))
/Users/emmaknippel/Desktop/POLIT 10 SEM/Dynamic Programming/Term Paper/ReplicatingInnov

Warning: Error fun1: Vprime must have 6480 elements